In [1]:
import pandas as pd
import numpy as np
import datetime
from datetime import date
import scipy.stats as st
import sympy as sp
import scipy.integrate as si
import math

In [2]:
company_dataset = pd.read_csv("Data/CRSP_stocks_clean.csv")

/var/folders/9x/zgzyj9l153v5qjjz_hdvd31r0000gn/T/ipykernel_48418/877988600.py:1: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  company_dataset = pd.read_csv("Data/CRSP_stocks_clean.csv")


In [3]:
#to change when we have more recent data
today = pd.to_datetime(date(2022, 12, 30))
#create historical mean starting date, should be about 3 years from current
historicalstart = (pd.to_datetime(date(2019,12,30)))
#pre-set lambda value
Lambda = 0.97
alpha = 0.05

In [4]:
def ConditionalMean(returns,startdate):
    #filter rows where date is after startdate
    conditionalMeanDF = returns[returns['date'] > startdate]
    #take the mean return
    conditionalmean = conditionalMeanDF["RET"].mean()
    #print('this is cond mean')
   # print(conditionalmean)
    return conditionalmean

def VolatilityDF(returns):
   #Calculate standard deviation of returns
    volatilitydf = returns
    volatilitydf.set_index('date',inplace=True)
    volatilitydf["Variance"] = volatilitydf.rolling(window=22).var()
    volatilitydf["Std"] = np.sqrt(volatilitydf["Variance"])
    volatilitydf = volatilitydf.reset_index()
    return(volatilitydf)

def ConditionalVariance(volatilitydf,today):
    yesterday = volatilitydf[volatilitydf["date"] == (today- datetime.timedelta(days=1))]
    today = volatilitydf[volatilitydf["date"] == today]
    yesterdayReturn = yesterday.iloc[0]["RET"]
    yesterdayVariance = yesterday.iloc[0]["Variance"]
    conditionalstd = math.sqrt((Lambda*yesterdayVariance)+ (1-Lambda)*(yesterdayReturn**2))
   # print('this is cond std')
   # print(conditionalstd)
    return conditionalstd

pi = math.pi

def ZScoreFunction(x):
    ZScore = (((1 / math.sqrt(2*pi)) * math.exp(-x**2/2))*x/0.01)
    return ZScore


def ExpZScore():
    lower_limit = -sp.oo  # Negative infinity
    upper_limit = st.norm.ppf(alpha)
    expectedZscore = si.quad(ZScoreFunction,lower_limit, upper_limit)[0]
   # print("this is expected value of zscore")
   # print(expectedZscore)
    return(expectedZscore)  



In [5]:
# groupy by ticker
grouped_data = company_dataset.groupby('TICKER')
returns = pd.DataFrame({})
for ticker,data in grouped_data:
    #creating a returns dataframe for each ticker with return and date columns, then indexing by date
    returns = data[['RET','date']].dropna()
    returns['date'] = pd.to_datetime(returns['date'])
    if (returns['date']==today).any():
        print('ticker ')
        print(ticker)
        returns.set_index('date',inplace=True)
        returns = returns.rolling(window=22).mean()
        returns = returns.reset_index()
        condmean = ConditionalMean(returns,historicalstart)
        volatilitydf = VolatilityDF(returns)
        conditionalstd = ConditionalVariance(volatilitydf,today)
        expzscore = ExpZScore()
        expectedshortfall = condmean + conditionalstd * expzscore
        print('expected shortfall')
        print(expectedshortfall)
    else:
        pass

ticker 
A
expected shortfall
-0.028520042654248314
ticker 
AAIC
expected shortfall
-0.017359399379080223
ticker 
AAME
expected shortfall
-0.039588992023305
ticker 
AAON
expected shortfall
-0.05092591652508836
ticker 
AAPL
expected shortfall
-0.03196086207336135
ticker 
AB
expected shortfall
-0.05418706624320635
ticker 
ABC
expected shortfall
-0.012853593949617428
ticker 
ABCB
expected shortfall
-0.02616376107696583
ticker 
ABEV
expected shortfall
-0.019000406031632618
ticker 
ABIO
expected shortfall
-0.03759252193550258
ticker 
ABM
expected shortfall
-0.015038480377223901
ticker 
ACGL
expected shortfall
-0.013114115292197037
ticker 
ACHV
expected shortfall
-0.07181699533752126
ticker 
ACIW
expected shortfall
-0.03458930943675485
ticker 
ADBE
expected shortfall
-0.026293890477012492
ticker 
ADC
expected shortfall
-0.009018950079594466
ticker 
ADI
expected shortfall
-0.03624605389108342
ticker 
ADM
expected shortfall
-0.010170521199942556
ticker 
ADP
expected shortfall
-0.033446275683072